### This is a second notebook due to environment issues

In [39]:
import numpy as np
import lab3_tools as tools
import lab3_proto as proto
import lab1_proto as proto1
import lab1_tools as tools1
import lab2_proto as proto2
import lab2_tools as tools2
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.utils import np_utils
from keras.utils import np_utils

In [40]:
stateList = np.load("stateList.npy", allow_pickle=True).tolist()

# 4.6 continuation

## Data conversion

In [41]:
output_dim = len(stateList)
#----
trainFile = np.load("flatDataTrain.npz")
lmfcc_train_x=trainFile["lmfcc_train_x"].astype('float32')
mspec_train_x=trainFile["mspec_train_x"].astype('float32')
dlmfcc_train_x=trainFile["dlmfcc_train_x"].astype('float32')
dmspec_train_x=trainFile["dmspec_train_x"].astype('float32')
train_y=np_utils.to_categorical(trainFile["train_y"],output_dim)
#print(lmfcc_train_x.shape,mspec_train_x.shape,dlmfcc_train_x.shape,dmspec_train_x.shape,train_y.shape) #| Shapes checked and OK
#----
valFile = np.load("flatDataVal.npz")
lmfcc_val_x=valFile["lmfcc_val_x"].astype('float32')
mspec_val_x=valFile["mspec_val_x"].astype('float32')
dlmfcc_val_x=valFile["dlmfcc_val_x"].astype('float32')
dmspec_val_x=valFile["dmspec_val_x"].astype('float32')
val_y=np_utils.to_categorical(valFile["val_y"],output_dim)
#print(lmfcc_val_x.shape,mspec_val_x.shape,dlmfcc_val_x.shape,dmspec_val_x.shape,val_y.shape)
#----
testFile = np.load("flatDataTest.npz")
lmfcc_test_x=testFile["lmfcc_test_x"].astype('float32')
mspec_test_x=testFile["mspec_test_x"].astype('float32')
dlmfcc_test_x=testFile["dlmfcc_test_x"].astype('float32')
dmspec_test_x=testFile["dmspec_test_x"].astype('float32')
test_y=np_utils.to_categorical(testFile["test_y"],output_dim)
#print(lmfcc_test_x.shape,mspec_test_x.shape,dlmfcc_test_x.shape,dmspec_test_x.shape,test_y.shape)

# 5. Phoneme Recognition with Deep Neural Networks

In [42]:
features = lmfcc_train_x
labels = train_y
features_test = lmfcc_test_x
labels_test = test_y
feature_dim = features.shape[1]
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(80,activation=tf.nn.relu,input_shape=(feature_dim,))) #choose ReLu since it's faster to compute compared to Sigmoid
model.add(tf.keras.layers.Dense(output_dim,activation=tf.nn.softmax)) #softmax distributes probabilitites across our states

model.compile(loss="categorical_crossentropy",
            optimizer="sgd",
            metrics=["accuracy"])

model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 80)                1120      
_________________________________________________________________
dense_11 (Dense)             (None, 61)                4941      
Total params: 6,061
Trainable params: 6,061
Non-trainable params: 0
_________________________________________________________________


In [43]:
EPOCHS = 20 #Goes through data set EPOCHS number of times 
BATCH_SIZE = 256 #Each training step model will see BATCH_SIZE number of examples to guide and adjust parameters

model.fit(features,labels,epochs=EPOCHS,batch_size=BATCH_SIZE)

Epoch 1/20
5264/5264 [==============================] - 5s 813us/step - loss: 2.5109 - accuracy: 0.3621
Epoch 2/20
5264/5264 [==============================] - 5s 867us/step - loss: 1.8401 - accuracy: 0.4691
Epoch 3/20
5264/5264 [==============================] - 5s 880us/step - loss: 1.7456 - accuracy: 0.4835
Epoch 4/20
5264/5264 [==============================] - 5s 878us/step - loss: 1.7078 - accuracy: 0.4904
Epoch 5/20
5264/5264 [==============================] - 5s 873us/step - loss: 1.6843 - accuracy: 0.4952
Epoch 6/20
5264/5264 [==============================] - 5s 887us/step - loss: 1.6664 - accuracy: 0.4990
Epoch 7/20
5264/5264 [==============================] - 5s 885us/step - loss: 1.6515 - accuracy: 0.5022
Epoch 8/20
5264/5264 [==============================] - 5s 892us/step - loss: 1.6385 - accuracy: 0.5049
Epoch 9/20
5264/5264 [==============================] - 5s 888us/step - loss: 1.6271 - accuracy: 0.5075
Epoch 10/20
5264/5264 [==============================] - 5s 894u

### Purpose of validation data? 

- Training data: Used to created weights of neural network
- Validation data: Used to compare different neural networks and choose the one that gives best result
- Test data: Simply used to test performance of the trained and chosen neural network. If not good results, start over. 

If you use the training and validation data as the same, the choice of neural network can be biased. Which means that the neural network was chosen as better just for that data. Real world data mighjt not have that connection to the neural network and might therefore not perform well. 

In [44]:
test_loss, test_acc = model.evaluate(features_test,labels_test)
print("test_acc:",test_acc) #Can increase with more epochs and more sofisticated model
print("test_loss:",test_loss)

47720/47720 [==============================] - 25s 526us/step - loss: 1.5896 - accuracy: 0.5108
test_acc: 0.5108054280281067
test_loss: 1.5896146297454834


## 5.2 Possible questions

- what is the influence of feature kind and size of input context window?
    - The input kind is important since that reveals the amount of "hidden data". Ex. the lmfcc has been stripped of some information compared to the mspecs. This can later be interpred by the hidden layers. 
    -   input context window $\color{red}{!!!}$
- what is the purpose of normalising (standardising) the input feature vectors depending on
the activation functions in the network?
    - Because otherwise some values that shouldn't activate the activation function will do so and vice versa. When normalising all values, one ensures that all data is represented in relation to each other.
    - Looking at what happens when one only normalises for each utterance, some parts of an utterance might trigger the activation function even though it wouldn't have if one normalised all utterances in relation to each other. This is why shorter utterances might be over represented when one only normalises for each utternace in isolation.
- what is the influence of the number of units per layer and the number of layers?
    - answer
- what is the influence of the activation function (when you try other activation functions
than ReLU, you do not need to reach convergence in case you do not have enough time)
    - answer
- what is the influence of the learning rate/learning rate strategy?
    - answer
- how stable are the posteriograms from the network in time?
    - answer
- how do the errors distribute depending on phonetic class?
    - answer